# Задание 1. A/B–тестирование

1.1 Условие

Одной из основных задач аналитика в нашей команде является корректное проведение экспериментов. 

Для этого мы применяем метод A/B–тестирования. 

В ходе тестирования одной гипотезы целевой группе была предложена новая механика оплаты услуг на сайте, 

у контрольной группы оставалась базовая механика. В качестве задания вам необходимо проанализировать итоги эксперимента 

и сделать вывод, стоит ли запускать новую механику оплаты на всех пользователей.

# Входные данные

В качестве входных данных мы имеем 4 csv-файла:

1. groups.csv – файл с информацией о принадлежности пользователя к контрольной или экспериментальной группе 
   (А – контроль, B – целевая группа) 
2. groups_add.csv – дополнительный файл с пользователями, который вам прислали спустя 2 дня после передачи данных
3. active_studs.csv – файл с информацией о пользователях, которые зашли на платформу в дни проведения эксперимента. 
4. checks.csv – файл с информацией об оплатах пользователей в дни проведения эксперимента. 

In [7]:
import pandas as pd
import numpy as np
import pandahouse as ph
import seaborn as sns
import matplotlib.pyplot as plt
import os
import scipy.stats as ss
from scipy.stats import shapiro
from scipy.stats import ttest_ind
import pandas as pd
import requests
from urllib.parse import urlencode
from io import StringIO

# EDA анализ

In [8]:
def read_yandex_disk(public_key, sep=','):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 

    final_url = base_url + urlencode(dict(public_key=public_key)) 
    response = requests.get(final_url) 
    download_url = response.json()['href'] 

    # загружаем файл
    response = requests.get(download_url)
    data = response.content.decode('utf-8')

    # читаем данные в DataFrame
    df = pd.read_csv(StringIO(data), sep=sep) 
    return df

In [10]:
groups        = read_yandex_disk('https://disk.yandex.ru/d/UhyYx41rTt3clQ', ';')
groups_add    = read_yandex_disk('https://disk.yandex.ru/d/5Kxrz02m3IBUwQ')
active_studs  = read_yandex_disk('https://disk.yandex.ru/d/Tbs44Bm6H_FwFQ')
checks        = read_yandex_disk('https://disk.yandex.ru/d/pH1q-VqcxXjsVA', ';')

In [11]:
groups.info()
groups.nunique()
# проверка на дупликаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74484 entries, 0 to 74483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      74484 non-null  int64 
 1   grp     74484 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


id     74484
grp        2
dtype: int64

In [140]:
groups_add.info()
groups_add.nunique()
# проверка на дупликаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      92 non-null     int64 
 1   grp     92 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


id     92
grp     2
dtype: int64

In [141]:
active_studs.info()
active_studs.nunique()
# проверка на дупликаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8341 entries, 0 to 8340
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   student_id  8341 non-null   int64
dtypes: int64(1)
memory usage: 65.3 KB


student_id    8341
dtype: int64

In [142]:
checks.info()
checks.nunique()
# проверка на дупликаты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   student_id  541 non-null    int64  
 1   rev         541 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 8.6 KB


student_id    541
rev            39
dtype: int64

In [143]:
full_groups = pd.concat([groups, groups_add])
full_groups
# добавляем дополнительных пользователей

,id,grp
0,1489,B
1,1627,A
2,1768,B
3,1783,B
4,1794,A
...,...,...
87,200247820,B
88,201032527,B
89,201067612,B
90,201067653,B


In [144]:
df_1 = active_studs.merge(checks, how = 'left', on = 'student_id').fillna(0)
df_1 = df_1.rename(columns={'student_id': 'id'})
df_1
# Так как у нас есть файл в котором указано "файл с пользователями, которые зашли именно в дни проведения эксперимента", 
# сделаем 'left join' по этому файлу.

,id,rev
0,581585,0.0
1,5723133,0.0
2,3276743,0.0
3,4238589,0.0
4,4475369,0.0
...,...,...
8336,1794344,0.0
8337,296883,0.0
8338,3598448,0.0
8339,3610547,290.0


In [145]:
dataset        = full_groups.merge(df_1, on = 'id')
dataset        = dataset.rename(columns={'grp' : 'not_active'})
dataset['buy'] = np.where(dataset.rev == 0, 0, 1)
dataset
# добавим колонку, в которой будет указано, была ли оплата

,id,not_active,rev,buy
0,1627,A,990.0,1
1,2085,B,0.0,0
2,2215,B,0.0,0
3,3391,A,0.0,0
4,3401,B,0.0,0
...,...,...,...,...
8336,5760471,B,0.0,0
8337,5777081,B,0.0,0
8338,5870856,A,0.0,0
8339,5945085,B,0.0,0


In [146]:
dataset_2 = dataset.query('buy > 0')
dataset_2 = dataset_2.rename(columns={'not_active' : 'active'})
dataset_2
# таблица только с теми, кто оплатил в дни проведения эксперимента

,id,active,rev,buy
0,1627,A,990.0000,1
10,100300,B,990.0000,1
12,108270,B,290.0000,1
246,264473,B,1900.0001,1
279,274352,B,690.0000,1
...,...,...,...,...
8312,5645737,B,1900.0000,1
8314,5651566,B,1900.0000,1
8316,5662322,B,1900.0000,1
8323,5681918,B,1900.0000,1


В итоге мы имеем количество оплативших в дни эксперимента, разделение на группы, суммы оплат.

В связи с этим, далее я принимаю решение рассчитывать конверсию, ARPU и ARPPU.

P.S. Замечаем, что просто оплативших пользователей в дни эксперемента из файла checks - 541, а тех, кто проявил активность и оплатил 392. Соотвественно, мы имеем 149 пользователей, которые оплатили, но не проявили активности на сайте. Возможно, стоит проверить их на баги/автооплату.

Далее учитывать их не будем.

# Расчёт CR

In [147]:
dataset_3       = pd.concat([dataset.not_active.value_counts(), dataset_2.active.value_counts()], axis = 1)
dataset_3       = dataset_3.reset_index()
dataset_3       = dataset_3.rename(columns={'index' : 'group'})
dataset_3['CR'] = (dataset_3.active/dataset_3.not_active)
dataset_3

,group,not_active,active,CR
0,B,6803,314,0.046156
1,A,1538,78,0.050715


Мы видим, что CR в группе B ниже, чем в группе A. Так как у нас используются номинативные переменные(есть оплата и нет оплаты), то воспользуемся статистическим тестом - критерием хи-квадрата.

H0: Новая механика оплаты услуг не повлияла на CR

H1: Новая механика оплаты услуг повлияла на CR

In [148]:
stat, p, expected, dof = ss.chi2_contingency(pd.crosstab(dataset.not_active, dataset.buy))

In [149]:
 stat, p

(0.48480826522557896, 0.48625240457231045)

Мы не отклоняем H0, т.к. p-значение не меньше 0,05. => статистически значимых различий нет.
В связи с этим мы не можем утверждать, что новая система оплаты успешна.

# Расчёт ARPU и ARPPU

In [150]:
dataset_count                 = dataset.groupby(['not_active'], as_index = False) \
.agg({'rev' : 'sum'})\
.rename(columns={'not_active' : 'group'})
dataset_count ['count_buy']   = dataset_3['active']
dataset_count [ 'count_all']  = dataset_3['not_active']
dataset_count ['count_buy']   = dataset_count['count_buy'].tolist()[::-1]
dataset_count ['count_all']   = dataset_count['count_all'].tolist()[::-1]
dataset_count ['ARPU']        = dataset_count['rev']/dataset_count ['count_all']
dataset_count [ 'ARPPU']      = dataset_count['rev']/dataset_count ['count_buy']
dataset_count ['CR']          = dataset_3['CR'].tolist()[::-1]
dataset_count

,group,rev,count_buy,count_all,ARPU,ARPPU,CR
0,A,72820.0000,78,1538,47.347204,933.589744,0.050715
1,B,394974.0035,314,6803,58.058798,1257.878992,0.046156


In [151]:
((dataset_count['ARPU'][1] - dataset_count['ARPU'][0]) / dataset_count['ARPU'][0]) * 100

22.62349832959378

In [152]:
((dataset_count['ARPPU'][1] - dataset_count['ARPPU'][0]) / dataset_count['ARPPU'][0]) * 100

34.73573383545856

In [153]:
((dataset_count['CR'][1] - dataset_count['CR'][0]) / dataset_count['CR'][0]) * 100

-8.98962373311925

In [158]:
# Подводим промежуточные итоги:
# 1) Конверсия в  группе В меньше на 8.9% чем в группе А
# 2) ARPPU в группе В больше на 34.7%
# 3) ARPU группе В больше на 22.6%
# Итого средняя цена чека выросла, при меньшей конверсии

In [159]:
# Проверим, статистически значимы ли эти различия или нет (ARPPU) при помощи Т-теста(который подходит для средних значений)
# Н0 - новый вариант оплаты не повлиял на средний чек
# Н1 - новый вариант оплаты повлиял на средний чек

Сначала используем тест Левена для проверки дисперсий.

In [178]:
dataset_A = dataset_2.query("active == 'A'")['rev']
dataset_B = dataset_2.query("active == 'B'")['rev']

In [161]:
pvalue_ab_levene = ss.levene(dataset_A, dataset_B)

In [162]:
pvalue_ab_levene[1] < 0.05

False

In [163]:
# Дисперсия между двумя группами равна

Далее проверим, является ли распределение нормальным при помощи критерия Шапиро-Уилка

In [164]:
p_value_a_shapiro = shapiro(dataset_A)

In [165]:
p_value_a_shapiro[1] < 0.05

True

In [166]:
p_value_b_shapiro = shapiro(dataset_B)

In [167]:
p_value_b_shapiro[1] < 0.05

True

In [168]:
# Распределение не является нормальным

In [ ]:
# P.S. Сначала, я хотел завернуть ттест, т.к. распределение не является нормальным, однако, в процессе ознакомления со статьёй 
# на habr (https://habr.com/ru/companies/glowbyte/articles/594183/), я выяснил, что "даже при маленьких выборках с ненормальным 
# распределением тест Стьюдента надёжен". Поэтому было принято решение завершить ттест.

In [199]:
ttest = ss.ttest_ind(dataset_A, dataset_B)

In [201]:
ttest[1] < 0.05

True

Поскольку p-значение меньше 0,05, мы отклоняем нулевую гипотезу.

Новый вариант оплаты повлиял на средний чек.

# ВЫВОД

В ходе анализа данных, мы рассмотрели такие метрики как CR, ARPU и ARPPU, т.к. это самые доступные и подходящие метрики исходя из тех данных, что нам предоставили.

Проведя тесты и анализ данных, мы может сделать следующие выводы:

Конверсия пользователей при новой системе оплаты изменилось статистически незначимо, что было подтверждено тестом.

Средний чек пользователей при новой механике оплаты статистически значимо вырос.

Несмотря на небольшое уменьшение конверсии, средний чек вырос, следовательно 
новая механика оплаты положительно влияет на итоговый результат.

# Задание 2. SQL

Задание 2.1

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

Условие:
    
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. 

Каждое такое маленькое задание называется "горошиной".Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

In [221]:
# параметры подключения
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [226]:
qqq = """

SELECT
  COUNT(st_id) as count_st
FROM
  (
    SELECT
      st_id,
      COUNT(correct) as count_correct,
      toYear(timest) AS year,
      toMonth(timest) AS month
    FROM
      default.peas
    WHERE
      correct = 1
    group by
      st_id,
      year,
      month
    HAVING
      count_correct >= 20
  ) as info
"""
dataset_students = ph.read_clickhouse(qqq, connection=connection_default)

Ответ: 136 усердных учеников

---------------------------------------------------------------------

Задание 2.1

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

In [246]:
aaa = """

WITH grp as (
  select
    distinct test_grp
  from
    default.studs
),
active_users as (
  SELECT
    DISTINCT test_grp,
    st_id AS st_id
  FROM
    (
      SELECT
        test_grp,
        st_id
      FROM
        default.peas
        join default.studs as st using (st_id)
      WHERE
        correct = 1
      GROUP BY
        test_grp,
        st_id
      HAVING
        count(correct) > 10
    )
),
active_users_agg as (
  SELECT
    test_grp,
    count(distinct(st_id)) as stds_id,
    sum(money) as money
  FROM
    active_users as au
    LEFT JOIN default.final_project_check ON au.st_id = default.final_project_check.st_id
  group by
    test_grp
),
count_studs_all as (
  SELECT
    test_grp,stds_id,money
  FROM
    (
      SELECT
        test_grp,COUNT(DISTINCT(default.studs.st_id)) as stds_id,
        SUM(money) as money
      FROM
        default.studs
        LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
        group by test_grp
    )
),
count_studs as (
  SELECT
    test_grp,stds_id
  FROM
    (
      SELECT
        test_grp,COUNT(DISTINCT(default.final_project_check.st_id)) as stds_id
      FROM
        default.studs
        LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
        group by test_grp
    )
),
count_studs_maths as (
  SELECT
    test_grp,stds_id
  FROM
    (
      SELECT
        test_grp,COUNT(DISTINCT(default.final_project_check.st_id)) as stds_id
      FROM
        default.studs
        LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
      WHERE
        default.final_project_check.subject = 'Math'
    group by test_grp
    )
),
active_math_users as (
  SELECT
    DISTINCT test_grp,st_id as st_id
  FROM
    (
      SELECT
        test_grp,st_id
      FROM
        default.peas
        join default.studs using  (st_id)
      WHERE
        correct = 1
        and subject = 'Math'
      GROUP BY
        test_grp,st_id
      HAVING
        COUNT(correct) >= 2
    )
),
active_math_users_agg as (
  SELECT
    test_grp,
    count(distinct(st_id)) as stds_id,
    sum(money) as money
  FROM
    active_math_users as au
    LEFT JOIN default.final_project_check ON au.st_id = default.final_project_check.st_id
  group by
    test_grp
)
SELECT
  grp.test_grp,
  ARPU,
  ARPAU,
  CR,
  CR_active,
  CR_math_active
from
  grp
  join (
    select
      test_grp,
      money/stds_id as ARPU
    from
      count_studs_all
  ) as ARPU ON ARPU.test_grp = grp.test_grp
  join (
    select
      test_grp,
      money/stds_id as ARPAU
    from
      active_users_agg
  ) as ARPAU ON ARPAU.test_grp = grp.test_grp
  join (
    select
      test_grp,
      COUNT(DISTINCT(default.final_project_check.st_id)) / COUNT(DISTINCT(default.studs.st_id)) as CR
    from
      default.studs
      LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
    group by
      test_grp
  ) as CR ON CR.test_grp = grp.test_grp
  JOIN
  (
    select
      test_grp,cs.stds_id/aug.stds_id as CR_active
    from
      active_users_agg as aug
      JOIN count_studs cs using (test_grp)
  ) as CR_active ON CR_active.test_grp = grp.test_grp
  JOIN
  (
    select
      test_grp,cs.stds_id/aug.stds_id as CR_math_active
    from
      active_math_users_agg as aug
      JOIN count_studs_maths cs using (test_grp)
  ) as CR_math_active ON CR_math_active.test_grp = grp.test_grp
"""

In [247]:
aaa = ph.read_clickhouse(aaa, connection=connection_default)
aaa

,grp.test_grp,ARPU,ARPAU,CR,CR_active,CR_math_active
0,pilot,11508.474576,29739.583333,0.111864,0.343750,0.380952
1,control,4540.983607,10393.700787,0.052459,0.125984,0.163265
